# Universidade de Brasília
## Programa de Pós-Graduação em Computação Aplicada
## Mestrato Profissional em Computação Aplicada
### Mineração de Dados
#### Professor: Marcelo Ladeira
#### Alunos: 
##### 22000xxxx - Alisson Melo Rios
##### 22000xxxx - Filipe Oliveira da Silva
##### 220005559 - Luís Eduardo Barreiro de Jesus


## PREPARAÇÃO DO AMBIENTE DE PROCESSAMENTO

In [1]:
# Preparação do Ambiente
## Seção de bibliotecas
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules

import json
import datetime 

import folium
from folium.plugins import HeatMap, HeatMapWithTime

from apyori import apriori

In [2]:
#import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from yellowbrick.classifier import ConfusionMatrix 

/Users/luisjesus/opt/anaconda3/lib/python3.9/site-packages/yellowbrick/style/colors.py:35: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  mpl_ge_150 = LooseVersion(mpl.__version__) >= "1.5.0"
/Users/luisjesus/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/luisjesus/opt/anaconda3/lib/python3.9/site-packages/yellowbrick/style/rcmod.py:31: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  mpl_ge_150 = LooseVersion(mpl.__version__) >= "1.5.0"
/Users/luisjesus/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


## MANIPULAÇÃO DE DADOS

Nesta seção, faz-se a manipulação dos dados de forma a se gerar as informações de acordo com a necessidade da análise dos dados.

### Colunas do Dataframe

Nesta subseção, trabalha-se a identificação de subgrupos de dados carregados de registros de Acidentes da PRF.

In [3]:
# Constantes
## Colunas do Dataframe e operações tipo 1
## sumarização da quantidade de pessoas e estado em consequência do acidente
col_oper1 = {'id':'count', 'pessoas':'sum', 'mortos':'sum', 'feridos_leves':'sum', 'feridos_graves':'sum', 'veiculos':'sum'}

## Colunas do Dataframe e operações tipo 2
col_oper2 = {'id':'count', 'mortos':'sum', 'feridos_leves':'sum', 'feridos_graves':'sum', 'veiculos':'sum'}

#------------------------------------------------------------
# Para a geração de subdatasets
#------------------------------------------------------------
## colunas dos registros de acidentes 
col_acidentes = ['id',  
                'data_inversa', 
                'dia_semana', 
                'horario', 
                'uf', 
                'br',
                'km',
                'municipio', 
                'causa_acidente', 
                'tipo_acidente',
                'classificacao_acidente', 
                'fase_dia', 
                'sentido_via',
                'condicao_metereologica', 
                'tipo_pista',
                'tracado_via', 
                'uso_solo',
                'latitude', 
                'longitude',
                'regional', 
                'delegacia', 
                'uop',
                'ilesos',
                'feridos_leves', 
                'feridos_graves', 
                'mortos',
                'ano_mes',
                'ano',
                'mes']

## colunas dos registros de veículos
col_veiculos = ['id',
                'id_veiculo', 
                'tipo_veiculo', 
                'marca', 
                'ano_fabricacao_veiculo',
                'ilesos',
                'feridos_leves', 
                'feridos_graves', 
                'mortos',
                'ano_mes',
                'ano',
                'mes']

## colunas dos registros de pessoas
col_pessoas =  ['id',
                'id_veiculo',
                'pesid',
                'tipo_envolvido', 
                'estado_fisico', 
                'idade', 
                'sexo']

### Carga do(s) arquivo(s) de registro de acidentes

A fonte de dados está disponível no sítio eletrônico de Dados Abertos da Polícia Rodoviária Federal, Órgão responsável, dentro outras, do patrulhamento das rodovias (CF 1988, art. 144, § 2º).

Link: https://www.gov.br/prf/pt-br/acesso-a-informacao/dados-abertos/dados-abertos-acidentes

In [4]:
# Carregar dados de acidentes - carregar de arquivos
df2021 = pd.read_csv("acidentes2021.csv", dtype={'br': 'str', 'id': 'int64'}, encoding="Latin1", sep=";")
#df2020 = pd.read_csv("acidentes2020.csv", dtype={'br': 'str', 'id': 'int64'}, encoding="Latin1", sep=";")

df = df2021[:]

#df=pd.concat([df2021, df2020])

#df['latitude']=df['latitude'].str.replace(",",".",count)
#df['longitude']=df['longitude'].str.replace(",",".",count)

In [5]:


# Manipulação de campo data - a partir da coluna data_inversa
#df2021["ano-mes"] = df2021["data_inversa"].str.slice(start=0, stop=7)
# Transformar o campo no tipo Datetime
df["data_inversa"] = pd.to_datetime(df["data_inversa"])
# Criar campo ano
df["ano"] = df["data_inversa"].dt.year
# Criar campo mês
df["mes"] = df["data_inversa"].dt.month
# Campo com ano e mês
df["ano_mes"] = df['data_inversa'].dt.strftime('%B/%Y')
# Manipulando o campo br
df["br"] = "BR-" + df["br"].str.zfill(3)

# print no dataframe para conferência
df

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,...,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop,ano,mes,ano_mes
0,331730,738258.0,2021-01-01,sexta-feira,05:30:00,SP,BR-116,453,REGISTRO,Reação tardia ou ineficiente do condutor,...,0,0,-24.561681,-47.867522,SR-SP,DEL05,UOP01,2021,1,January/2021
1,331804,738412.0,2021-01-01,sexta-feira,08:05:00,AM,BR-174,937,MANAUS,Reação tardia ou ineficiente do condutor,...,0,0,-2.508068,-60.036434,SR-AM,DEL01,UOP01,2021,1,January/2021
2,331815,738692.0,2021-01-01,sexta-feira,10:10:00,CE,BR-222,"128,5",ITAPAJE,Velocidade Incompatível,...,0,0,-3.707626,-39.623509,SR-CE,DEL04,UOP03,2021,1,January/2021
3,331815,738689.0,2021-01-01,sexta-feira,10:10:00,CE,BR-222,"128,5",ITAPAJE,Velocidade Incompatível,...,0,0,-3.707626,-39.623509,SR-CE,DEL04,UOP03,2021,1,January/2021
4,331823,740272.0,2021-01-01,sexta-feira,12:30:00,RJ,BR-493,18,MAGE,Ingestão de álcool e/ou substâncias psicoativa...,...,0,0,-22.668712,-43.018627,SR-RJ,DEL02,UOP04,2021,1,January/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150914,460688,995324.0,2021-08-13,sexta-feira,23:52:00,PR,BR-376,175,MARINGA,Velocidade Incompatível,...,0,0,-23.412375,-51.943388,SR-PR,DEL09,UOP01,2021,8,August/2021
150915,460941,995895.0,2021-08-10,terça-feira,06:20:00,BA,BR-110,176,RIBEIRA DO POMBAL,Manobra de mudança de faixa,...,0,0,-10.896299,-38.523520,SR-BA,DEL07,UOP02,2021,8,August/2021
150916,461175,996489.0,2021-11-14,domingo,07:00:00,GO,BR-070,"18,3",AGUAS LINDAS DE GOIAS,Manobra de mudança de faixa,...,1,0,-15.744741,-48.355934,SR-DF,DEL01,UOP03,2021,11,November/2021
150917,461175,996488.0,2021-11-14,domingo,07:00:00,GO,BR-070,"18,3",AGUAS LINDAS DE GOIAS,Manobra de mudança de faixa,...,0,0,-15.744741,-48.355934,SR-DF,DEL01,UOP03,2021,11,November/2021


In [6]:
# Montar a estrutura para montar o mapa com o histórico de acidentes
# >>> lista_tempo = []

#for x in df2021["mes"]:
#    mesinteiro = x
#    lista_tempo.append(datetime.date(1900, mesinteiro, 1).strftime('%B'))

#df2021["month_in_full"] = lista_tempo



# >>> dfbr = df2021["br"]
# >>> dfbr = dfbr.drop_duplicates(keep='last')
# >>> jsonbr = dfbr.to_json(orient="split")

#print(lista_rod)
#dfbr = df2021.iloc[:,["br"]]

#lista_br = []
#for i in lista_rod:
#    lista_br.append(i)

#print(lista_br)
#jsonbr = json.dumps(dfbr(_dict_))
                                      
#print(jsonbr)
#dfbr

#folium.LayerControl().add_to(jsonbr)

In [7]:
# criar geopandas
#br = gpd.read_file(r"PATH_FILE")
#br.plot()


In [8]:
# Seção de filtros - pegar somente os acidentes com vítimas - Fatais ou não
#df2021 = df2021.loc[(df2021['classificacao_acidente']=='Com Vítimas Fatais')]

In [9]:
df

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,...,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop,ano,mes,ano_mes
0,331730,738258.0,2021-01-01,sexta-feira,05:30:00,SP,BR-116,453,REGISTRO,Reação tardia ou ineficiente do condutor,...,0,0,-24.561681,-47.867522,SR-SP,DEL05,UOP01,2021,1,January/2021
1,331804,738412.0,2021-01-01,sexta-feira,08:05:00,AM,BR-174,937,MANAUS,Reação tardia ou ineficiente do condutor,...,0,0,-2.508068,-60.036434,SR-AM,DEL01,UOP01,2021,1,January/2021
2,331815,738692.0,2021-01-01,sexta-feira,10:10:00,CE,BR-222,"128,5",ITAPAJE,Velocidade Incompatível,...,0,0,-3.707626,-39.623509,SR-CE,DEL04,UOP03,2021,1,January/2021
3,331815,738689.0,2021-01-01,sexta-feira,10:10:00,CE,BR-222,"128,5",ITAPAJE,Velocidade Incompatível,...,0,0,-3.707626,-39.623509,SR-CE,DEL04,UOP03,2021,1,January/2021
4,331823,740272.0,2021-01-01,sexta-feira,12:30:00,RJ,BR-493,18,MAGE,Ingestão de álcool e/ou substâncias psicoativa...,...,0,0,-22.668712,-43.018627,SR-RJ,DEL02,UOP04,2021,1,January/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150914,460688,995324.0,2021-08-13,sexta-feira,23:52:00,PR,BR-376,175,MARINGA,Velocidade Incompatível,...,0,0,-23.412375,-51.943388,SR-PR,DEL09,UOP01,2021,8,August/2021
150915,460941,995895.0,2021-08-10,terça-feira,06:20:00,BA,BR-110,176,RIBEIRA DO POMBAL,Manobra de mudança de faixa,...,0,0,-10.896299,-38.523520,SR-BA,DEL07,UOP02,2021,8,August/2021
150916,461175,996489.0,2021-11-14,domingo,07:00:00,GO,BR-070,"18,3",AGUAS LINDAS DE GOIAS,Manobra de mudança de faixa,...,1,0,-15.744741,-48.355934,SR-DF,DEL01,UOP03,2021,11,November/2021
150917,461175,996488.0,2021-11-14,domingo,07:00:00,GO,BR-070,"18,3",AGUAS LINDAS DE GOIAS,Manobra de mudança de faixa,...,0,0,-15.744741,-48.355934,SR-DF,DEL01,UOP03,2021,11,November/2021


### Dataframe dos Acidentes

In [10]:
# Criar Dataframe com os registros de acidentes (a estrutura do arquivo tem repetição nesses campos)
dfacid = df[col_acidentes]
# Total de registros antes de retirada das duplicações
len(dfacid)

150919

OBS:
- O total de registros do período é 150.919

In [11]:
# retirar as duplicações do acidentes
dfacid = dfacid.drop_duplicates(subset=['id'], keep='last')
# Total de registros únicos
len(dfacid)

64536

OBS:
- O total de acidentes do período é de 64.536;

### Estatística Descritiva dos dados de Acidentes

In [12]:
# imprimir os dados para verificação visual
dfacid.head(5)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,regional,delegacia,uop,ilesos,feridos_leves,feridos_graves,mortos,ano_mes,ano,mes
0,331730,2021-01-01,sexta-feira,05:30:00,SP,BR-116,453,REGISTRO,Reação tardia ou ineficiente do condutor,Tombamento,...,SR-SP,DEL05,UOP01,0,1,0,0,January/2021,2021,1
6,331844,2021-01-01,sexta-feira,15:20:00,BA,BR-101,"818,1",ITAMARAJU,Velocidade Incompatível,Tombamento,...,SR-BA,DEL09,UOP02,0,1,0,0,January/2021,2021,1
8,331857,2021-01-01,sexta-feira,15:30:00,GO,BR-020,183,ALVORADA DO NORTE,Demais falhas mecânicas ou elétricas,Incêndio,...,SR-DF,DEL02,UOP02,1,0,0,0,January/2021,2021,1
16,331920,2021-01-01,sexta-feira,20:40:00,PR,BR-116,59,CAMPINA GRANDE DO SUL,Entrada inopinada do pedestre,Atropelamento de Pedestre,...,SR-PR,DEL01,UOP02,0,0,0,1,January/2021,2021,1
17,331925,2021-01-01,sexta-feira,20:30:00,BA,BR-116,709,MANOEL VITORINO,Demais falhas mecânicas ou elétricas,Incêndio,...,SR-BA,DEL03,UOP01,1,0,0,0,January/2021,2021,1


SUGESTÕES DE PERGUNTAS:
- Relação de causa do acidente x tipo do acidente x horario do acidente x condição meteorologica x tipo de pista x traçado da via x uso do solo x dia da semana com a mortalidade dos acidentes?
- Relação de tipos de veículos x mortalidade dos acidentes?
- Escolher um trecho de rodovia concedida para analisar. Exemplo: BR-040 DF até Juiz de Fora; 

#### Domínios do acidente

In [13]:
#lista = dfacid.causa_acidente.unique()
#lista = dfacid.tipo_acidente.unique()
#lista = dfacid.classificacao_acidente.unique()
#lista = dfacid.fase_dia.unique()
#lista = dfacid.sentido_via.unique()
#lista = dfacid.condicao_metereologica.unique()
#lista = dfacid.tipo_pista.unique()
#lista = dfacid.tracado_via.unique()
#lista = dfacid.uso_solo.unique()
#print(', '.join(lista))

Exemplo   | Valor do exemplo
--------- | ------
causa_acidente | Reação tardia ou ineficiente do condutor ,Velocidade Incompatível ,Demais falhas mecânicas ou elétricas ,Entrada inopinada do pedestre ,Condutor Dormindo ,Frear bruscamente ,Acessar a via sem observar a presença dos outros veículos ,Chuva ,Ausência de reação do condutor ,Pedestre andava na pista ,Problema com o freio ,Carga excessiva e/ou mal acondicionada ,Ingestão de álcool pelo condutor ,Acostamento em desnível ,Ultrapassagem Indevida ,Mal súbito do condutor ,Curva acentuada ,Avarias e/ou desgaste excessivo no pneu ,Acumulo de água sobre o pavimento ,Condutor deixou de manter distância do veículo da frente ,Pedestre cruzava a pista fora da faixa ,Falta de elemento de contenção que evite a saída do leito carroçável ,Manobra de mudança de faixa ,Demais falhas na via ,Condutor usando celular ,Pista Escorregadia ,Ingestão de álcool ou de substâncias psicoativas pelo pedestre ,Obstrução na via ,Ingestão de álcool e/ou substâncias psicoativas pelo pedestre ,Acumulo de óleo sobre o pavimento ,Ingestão de substâncias psicoativas pelo condutor ,Transitar na contramão ,Iluminação deficiente ,Pista esburacada ,Conversão proibida ,Afundamento ou ondulação no pavimento ,Animais na Pista ,Demais Fenômenos da natureza ,Transitar no acostamento ,Desrespeitar a preferência no cruzamento ,Falta de acostamento ,Objeto estático sobre o leito carroçável ,Pista em desnível ,Estacionar ou parar em local proibido ,Área urbana sem a presença de local apropriado para a travessia de pedestres ,Desvio temporário ,Declive acentuado ,Obras na pista ,Ausência de sinalização ,Acumulo de areia ou detritos sobre o pavimento ,Problema na suspensão ,Acesso irregular ,Retorno proibido ,Sinalização mal posicionada ,Deficiência do Sistema de Iluminação/Sinalização ,Faixas de trânsito com largura insuficiente ,Restrição de visibilidade em curvas horizontais ,Neblina ,Fumaça ,Faróis desregulados ,Sistema de drenagem ineficiente ,Restrição de visibilidade em curvas verticais ,Condutor desrespeitou a iluminação vermelha do semáforo ,Trafegar com motocicleta (ou similar) entre as faixas ,Redutor de velocidade em desacordo ,Deixar de acionar o farol da motocicleta (ou similar) ,Transitar na calçada ,Semáforo com defeito ,Participar de racha ,Sinalização encoberta ,Modificação proibida
tipo_acidente | Tombamento ,Incêndio ,Atropelamento de Pedestre ,Saída de leito carroçável ,Capotamento ,Colisão com objeto ,Colisão traseira ,Derramamento de carga ,Colisão lateral ,Eventos atípicos ,Colisão frontal ,Colisão lateral sentido oposto ,Colisão lateral mesmo sentido ,Atropelamento de Animal ,Colisão transversal ,Engavetamento ,Queda de ocupante de veículo
classificacao_acidente | Com Vítimas Feridas, Sem Vítimas, Com Vítimas Fatais
fase_dia | Amanhecer, Pleno dia, Plena Noite, Anoitecer
sentido_via | Crescente, Decrescente, Não Informado
condicao_metereologica | Nublado, Céu Claro, Chuva, Ignorado, Sol, Garoa/Chuvisco, Nevoeiro/Neblina, Vento, Neve, Granizo
tipo_pista | Dupla, Simples, Múltipla
tracado_via | Curva, Reta, Não Informado, Desvio Temporário, Retorno Regulamentado, Ponte, Interseção de vias, Rotatória, Túnel, Viaduto
uso_solo | Não, Sim



In [14]:
dictio = {'id':'count', 'ilesos':'sum','feridos_leves':'sum','feridos_graves':'sum', 'mortos':'sum'}

# Análise estatística dos dados de acidentes
def _acidentes(df, aggr_field, dicio ):
    ## Acidentes por ano_mes
    #dfagg_acid = dfacid.groupby(['causa_acidente']).agg({'id':'count', 'ilesos':'sum','feridos_leves':'sum','feridos_graves':'sum', 'mortos':'sum'})
    df = df.groupby(aggr_field).agg(dicio)
    # Altera o nome da coluna id para qtd - count(id)
    df.rename(columns={'id':'qtd'}, inplace=True)
    df['percentual_mortalidade'] = df['mortos'] / df['qtd'] * 100
    df.reset_index(inplace=True)
    df = df.sort_values(by=['percentual_mortalidade'],ascending=False)
    
    return df
# imprimir
#dfagg_acid


def _acidente_pessoas(df, aggr_field, dict_pess):
    print(aggr_field)
    df = df.groupby(aggr_field).agg(dict_pess)
    return df

In [15]:
aggr_fld = ['causa_acidente']

df1 = _acidentes(dfacid, aggr_fld, dictio)
df1

,causa_acidente,qtd,ilesos,feridos_leves,feridos_graves,mortos,percentual_mortalidade
61,Sinalização encoberta,3,1,1,0,1,33.333333
66,Transitar na contramão,1780,489,467,373,357,20.056180
48,Pedestre andava na pista,700,310,97,84,126,18.000000
27,Entrada inopinada do pedestre,577,327,89,54,65,11.265165
68,Ultrapassagem Indevida,1649,503,595,305,179,10.855064
...,...,...,...,...,...,...,...
42,Modificação proibida,8,4,1,3,0,0.000000
32,Faróis desregulados,9,2,4,2,0,0.000000
34,Fumaça,37,27,6,0,0,0.000000
47,Participar de racha,7,2,1,3,0,0.000000


In [16]:
aggr_fld = ['tipo_acidente']
df1 = _acidentes(dfacid, aggr_fld, dictio)
df1

,tipo_acidente,qtd,ilesos,feridos_leves,feridos_graves,mortos,percentual_mortalidade
4,Colisão frontal,4346,1139,1192,1031,820,18.867925
1,Atropelamento de Pedestre,2910,1387,516,361,346,11.890034
7,Colisão lateral sentido oposto,1609,631,534,279,88,5.469236
12,Eventos atípicos,267,129,83,29,13,4.868914
15,Saída de leito carroçável,10652,2777,5314,1361,448,4.205783
3,Colisão com objeto,4907,1622,2064,624,202,4.116568
0,Atropelamento de Animal,978,258,448,209,38,3.885481
16,Tombamento,5572,953,3478,837,186,3.338119
2,Capotamento,1750,435,934,193,51,2.914286
8,Colisão transversal,8018,3683,2641,1185,221,2.756298


In [17]:
aggr_fld = ['classificacao_acidente']
df1 = _acidentes(dfacid, aggr_fld, dictio)
df1

,classificacao_acidente,qtd,ilesos,feridos_leves,feridos_graves,mortos,percentual_mortalidade
0,Com Vítimas Fatais,4663,822,343,371,2922,62.663521
1,Com Vítimas Feridas,48184,13786,25217,8102,0,0.000000
2,Sem Vítimas,11689,9774,0,0,0,0.000000


In [18]:
aggr_fld = ['fase_dia']
df1 = _acidentes(dfacid, aggr_fld, dictio)
df1

,fase_dia,qtd,ilesos,feridos_leves,feridos_graves,mortos,percentual_mortalidade
0,Amanhecer,2998,1004,1197,388,190,6.337558
2,Plena Noite,23099,8392,8348,3289,1384,5.991601
1,Anoitecer,3654,1429,1389,516,144,3.940887
3,Pleno dia,34785,13557,14626,4280,1204,3.461262


In [19]:
aggr_fld = ['tipo_pista']
df1 = _acidentes(dfacid, aggr_fld, dictio)
df1

,tipo_pista,qtd,ilesos,feridos_leves,feridos_graves,mortos,percentual_mortalidade
2,Simples,31804,11242,12192,4840,2014,6.332537
0,Dupla,27228,10790,11150,3076,783,2.875716
1,Múltipla,5504,2350,2218,557,125,2.271076


In [20]:
aggr_fld = ['tracado_via']
df1 = _acidentes(dfacid, aggr_fld, dictio)
df1

,tracado_via,qtd,ilesos,feridos_leves,feridos_graves,mortos,percentual_mortalidade
3,Não Informado,8268,3038,3246,1157,456,5.515239
0,Curva,9811,2975,4539,1358,533,5.432678
4,Ponte,364,134,126,66,19,5.219780
1,Desvio Temporário,1819,642,757,245,84,4.617922
5,Reta,39053,15314,14938,5025,1722,4.409392
2,Interseção de vias,2713,1243,955,323,64,2.359012
8,Túnel,91,28,42,15,2,2.197802
9,Viaduto,458,187,172,59,10,2.183406
7,Rotatória,1179,488,484,123,25,2.120441
6,Retorno Regulamentado,780,333,301,102,7,0.897436


In [21]:
aggr_fld = ['uso_solo']
df1 = _acidentes(dfacid, aggr_fld, dictio)
df1

,uso_solo,qtd,ilesos,feridos_leves,feridos_graves,mortos,percentual_mortalidade
0,Não,35811,12235,14796,4919,2162,6.037251
1,Sim,28725,12147,10764,3554,760,2.645779


In [22]:
# Plotar o gráfico
#dfacid.plot(kind='bar', x='br', y='qtd')


In [23]:
# Análise estatística dos dados de acidentes
## Acidentes por UF
dfacid.describe()

,id,latitude,longitude,ilesos,feridos_leves,feridos_graves,mortos,ano,mes
count,64536.000000,64536.000000,64536.000000,64536.000000,64536.000000,64536.000000,64536.000000,64536.0,64536.000000
mean,369713.321278,-18.940904,-46.669323,0.377805,0.396058,0.131291,0.045277,2021.0,6.623265
std,21881.312254,7.781611,6.186200,0.484842,0.489081,0.337721,0.207913,0.0,3.445482
min,331693.000000,-33.689819,-72.119380,0.000000,0.000000,0.000000,0.000000,2021.0,1.000000
25%,351091.750000,-25.364531,-50.335396,0.000000,0.000000,0.000000,0.000000,2021.0,4.000000
50%,369783.500000,-20.466573,-47.867522,0.000000,0.000000,0.000000,0.000000,2021.0,7.000000
75%,388080.250000,-12.935087,-42.624258,1.000000,1.000000,0.000000,0.000000,2021.0,10.000000
max,461534.000000,4.461419,-34.828232,1.000000,1.000000,1.000000,1.000000,2021.0,12.000000


In [24]:
dfacid.longitude.mean()

-46.669323470420245

### Plotar Mapa de Calor com tempo dos acidentes

In [25]:
###
# criar lista de ano_mes únicos
lista_index = df.ano_mes.unique()

print(lista_index)

lista_idx = []
for i in lista_index:
    lista_idx.append(i)

print(lista_idx)
# -------------------------------------------

# criar a weight list
weight_list = []
dfacid['conta'] = 1
for x in dfacid['ano_mes'].sort_values().unique():
    weight_list.append(dfacid.loc[dfacid['ano_mes'] == x, ["latitude", "longitude", "conta"]].groupby(["latitude", "longitude"]).sum().reset_index().values.tolist()) 

# -------------------------------------------

# criar o mapa base
my_map = folium.Map(
                location=(
                            (dfacid.latitude.mean()),
                            (dfacid.longitude.mean())
                        ),
                zoom_start=6
            )

#folium.LayerControl().add_to(my_map)
folium.TileLayer('stamentoner').add_to(my_map)

# criar o mapa de calor de acidentes
HeatMapWithTime(weight_list, radius=10, index=lista_idx,
                gradient={0.1: 'blue', 0.25: 'yellow', 0.75:'orange', 1:'red'},
                auto_play=True, min_opacity=0.5, max_opacity=1, use_local_extrema=True).add_to(my_map)

my_map

['January/2021' 'February/2021' 'March/2021' 'April/2021' 'May/2021'
 'June/2021' 'July/2021' 'August/2021' 'September/2021' 'October/2021'
 'November/2021' 'December/2021']
['January/2021', 'February/2021', 'March/2021', 'April/2021', 'May/2021', 'June/2021', 'July/2021', 'August/2021', 'September/2021', 'October/2021', 'November/2021', 'December/2021']


### Dataframe dos Veículos

In [26]:
# Criar Dataframe com os veículos
dfveic = df[col_veiculos]
len(dfveic)

150919

In [27]:
dfveic

,id,id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo,ilesos,feridos_leves,feridos_graves,mortos,ano_mes,ano,mes
0,331730,595150,Caminhão,VOLVO/VM 260 6X2R/VM 260 6X2R,2010.0,0,1,0,0,January/2021,2021,1
1,331804,595262,Caminhão-trator,SCANIA/R112 H/R112 H,1982.0,1,0,0,0,January/2021,2021,1
2,331815,595291,Caminhão-trator,SCANIA/R124 GA4X2NZ 360/R124 GA4X2NZ 360,1999.0,0,1,0,0,January/2021,2021,1
3,331815,595291,Caminhão-trator,SCANIA/R124 GA4X2NZ 360/R124 GA4X2NZ 360,1999.0,0,1,0,0,January/2021,2021,1
4,331823,595234,Caminhão-trator,VOLVO/FH 460 6X2T/FH 460 6X2T,2017.0,1,0,0,0,January/2021,2021,1
...,...,...,...,...,...,...,...,...,...,...,...,...
150914,460688,804988,Automóvel,I/VW FOX 1.0 GII/VW FOX 1.0 GII,NaN,0,0,0,0,August/2021,2021,8
150915,460941,805418,Caminhonete,GM/CHEVROLET D20 CUSTOM S/CHEVROLET D20 CUSTOM S,1989.0,1,0,0,0,August/2021,2021,8
150916,461175,805872,Motocicleta,HONDA/CG 160 FAN/CG 160 FAN,2020.0,0,0,1,0,November/2021,2021,11
150917,461175,805871,Automóvel,FIAT/UNO MILLE EX/UNO MILLE EX,2000.0,0,0,0,0,November/2021,2021,11


In [28]:
# retirar as duplicações do veículos
dfveic = dfveic.drop_duplicates(subset=['id_veiculo'], keep='last')
# Total de registros únicos
len(dfveic)

106730

In [29]:
dfveic

,id,id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo,ilesos,feridos_leves,feridos_graves,mortos,ano_mes,ano,mes
0,331730,595150,Caminhão,VOLVO/VM 260 6X2R/VM 260 6X2R,2010.0,0,1,0,0,January/2021,2021,1
1,331804,595262,Caminhão-trator,SCANIA/R112 H/R112 H,1982.0,1,0,0,0,January/2021,2021,1
3,331815,595291,Caminhão-trator,SCANIA/R124 GA4X2NZ 360/R124 GA4X2NZ 360,1999.0,0,1,0,0,January/2021,2021,1
4,331823,595234,Caminhão-trator,VOLVO/FH 460 6X2T/FH 460 6X2T,2017.0,1,0,0,0,January/2021,2021,1
5,331843,595320,Caminhão-trator,M.BENZ/AXOR 2544 LS/AXOR 2544 LS,2019.0,1,0,0,0,January/2021,2021,1
...,...,...,...,...,...,...,...,...,...,...,...,...
150914,460688,804988,Automóvel,I/VW FOX 1.0 GII/VW FOX 1.0 GII,NaN,0,0,0,0,August/2021,2021,8
150915,460941,805418,Caminhonete,GM/CHEVROLET D20 CUSTOM S/CHEVROLET D20 CUSTOM S,1989.0,1,0,0,0,August/2021,2021,8
150916,461175,805872,Motocicleta,HONDA/CG 160 FAN/CG 160 FAN,2020.0,0,0,1,0,November/2021,2021,11
150917,461175,805871,Automóvel,FIAT/UNO MILLE EX/UNO MILLE EX,2000.0,0,0,0,0,November/2021,2021,11


In [30]:
aggr_fld = ['tipo_veiculo']
df1 = _acidentes(dfveic, aggr_fld, dictio)
df1

,tipo_veiculo,qtd,ilesos,feridos_leves,feridos_graves,mortos,percentual_mortalidade
6,Carro de mão,10,1,3,1,3,30.000000
14,Outros,382,36,22,32,60,15.706806
22,Triciclo,10,1,4,4,1,10.000000
1,Bicicleta,2007,55,1082,591,196,9.765820
9,Ciclomotor,269,15,157,70,16,5.947955
11,Motocicleta,23148,1579,13844,5739,1304,5.633316
7,Carroça-charrete,55,9,24,8,3,5.454545
16,Reboque,55,13,5,6,2,3.636364
12,Motoneta,2763,151,1814,655,99,3.583062
19,Trator de rodas,62,45,7,5,2,3.225806


### Dataframe das pessoas

In [31]:
# Criar Dataframe com os veículos
dfpess = df[col_pessoas]
dfpess

,id,id_veiculo,pesid,tipo_envolvido,estado_fisico,idade,sexo
0,331730,595150,738258.0,Condutor,Lesões Leves,35.0,Masculino
1,331804,595262,738412.0,Condutor,Ileso,65.0,Masculino
2,331815,595291,738692.0,Passageiro,Lesões Leves,NaN,Feminino
3,331815,595291,738689.0,Condutor,Lesões Leves,35.0,Masculino
4,331823,595234,740272.0,Condutor,Ileso,51.0,Masculino
...,...,...,...,...,...,...,...
150914,460688,804988,995324.0,Condutor,Não Informado,NaN,Não Informado
150915,460941,805418,995895.0,Condutor,Ileso,53.0,Masculino
150916,461175,805872,996489.0,Condutor,Lesões Graves,27.0,Masculino
150917,461175,805871,996488.0,Condutor,Não Informado,NaN,Não Informado


In [32]:
dfpess.describe()

,id,id_veiculo,pesid,idade
count,150919.000000,150919.000000,150918.000000,135648.000000
mean,370187.409816,667992.774018,826150.874356,39.606518
std,22153.763365,41854.895427,50079.619162,46.332658
min,331693.000000,595095.000000,738204.000000,0.000000
25%,351357.000000,632029.500000,783352.250000,27.000000
50%,370453.000000,668732.000000,826290.500000,37.000000
75%,388711.000000,703508.000000,868490.750000,48.000000
max,461534.000000,806594.000000,997402.000000,2020.000000


In [33]:
aggr_fld = ['tipo_envolvido', 'estado_fisico']
dict_pess={'tipo_envolvido':'count', 'idade':'mean'}
dfa = _acidente_pessoas(dfpess, aggr_fld, dict_pess)
dfa

['tipo_envolvido', 'estado_fisico']


tipo_envolvido      idade
tipo_envolvido estado_fisico                           
Cavaleiro      Ileso                       6  45.000000
               Lesões Graves               7  51.714286
               Lesões Leves               10  35.200000
               Não Informado               1        NaN
               Óbito                       2  33.500000
Condutor       Ileso                   50364  41.790771
               Lesões Graves           11572  38.565839
               Lesões Leves            34554  38.374416
               Não Informado            6831        NaN
               Óbito                    3382  40.867959
Não Informado  Não Informado               1        NaN
Passageiro     Ileso                   12919  33.392658
               Lesões Graves            4923  38.598050
               Lesões Leves            18561  39.973395
               Não Informado             442        NaN
               Óbito                    1118  36.734694
Pedestre       Ileso                      63  34.262295
               Lesões Graves            1161  42.051282
               Lesões Leves             1055  46.552941
               Não Informado              51        NaN
               Óbito                     893  48.742560
Testemunha     Não Informado            3003        NaN

## Random Forest

In [34]:
# Importação das bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [35]:
#df2021

colunas =  ['causa_acidente', 
            'tipo_acidente', 
            'fase_dia', 
            'condicao_metereologica', 
            'tipo_pista',
            'tracado_via',
            'classificacao_acidente']


df2rf = dfacid[colunas]



In [36]:
# Formato de matriz
previsores = df2rf.iloc[:,0:6].values
classe = df2rf.iloc[:,6].values
#classe2 = df2apriori.iloc[:,6].values


In [37]:
# Transformação dos atributos categóricos em atributos numéricos, passando o índice de cada atributo categórico
labelencoder = LabelEncoder()
previsores[:,0] = labelencoder.fit_transform(previsores[:,0])
previsores[:,1] = labelencoder.fit_transform(previsores[:,1])
previsores[:,2] = labelencoder.fit_transform(previsores[:,2])
previsores[:,3] = labelencoder.fit_transform(previsores[:,3])
previsores[:,4] = labelencoder.fit_transform(previsores[:,4])
previsores[:,5] = labelencoder.fit_transform(previsores[:,5])
#previsores[:,6] = labelencoder.fit_transform(previsores[:,6])
#previsores[:, 11] = labelencoder.fit_transform(previsores[:, 11])
#previsores[:, 13] = labelencoder.fit_transform(previsores[:, 13])
#previsores[:, 14] = labelencoder.fit_transform(previsores[:, 14])
#previsores[:, 16] = labelencoder.fit_transform(previsores[:, 16])
#previsores[:, 18] = labelencoder.fit_transform(previsores[:, 18])
#previsores[:, 19] = labelencoder.fit_transform(previsores[:, 19])

In [38]:
# Divisão da base de dados entre treinamento e teste (30% para testar e 70% para treinar)
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores,
                                                                  classe,
                                                                  test_size = 0.3,
                                                                  random_state = 0)

In [39]:
X_treinamento

array([[69, 14, 3, 1, 0, 5],
       [8, 9, 2, 1, 0, 5],
       [49, 1, 3, 8, 1, 5],
       ...,
       [64, 9, 3, 1, 0, 5],
       [14, 9, 2, 1, 0, 5],
       [69, 15, 3, 1, 2, 0]], dtype=object)

In [40]:
y_treinamento

array(['Com Vítimas Fatais', 'Com Vítimas Feridas', 'Com Vítimas Feridas',
       ..., 'Com Vítimas Feridas', 'Sem Vítimas', 'Com Vítimas Feridas'],
      dtype=object)

In [41]:
# Criação do modelo, treinamento, obtenção das previsões e da taxa de acerto
floresta = RandomForestClassifier(n_estimators = 100)
floresta.fit(X_treinamento, y_treinamento)

RandomForestClassifier()

In [42]:
# Visualização dos atributos principais
#floresta.estimators_
floresta.estimators_[1]

DecisionTreeClassifier(max_features='auto', random_state=351054681)

In [43]:
previsoes = floresta.predict(X_teste)
confusao = confusion_matrix(y_teste, previsoes)
taxa_acerto = accuracy_score(y_teste, previsoes)
taxa_acerto

0.735344248747482